## Playground for trying stuff here

In [5]:
import numpy as np
from timeit import default_timer as timer

In [13]:
def post_process_IBD(IBD_blocks, spacing=0.05):
    '''Post process IBD sharing. 
    Pool together IBD blocks that are less than 
    spacing cM apart. Update IBD block list'''

    ibd_list=IBD_blocks
    bl_list_final = [] # The pruned IBD List. Empty Container

    print("Starting Post Processing!")
    start0 = timer()
    # Update the End to absolute end of block:
    ibd_list = [(x[0], x[0] + x[1], x[2], x[3], x[4]) for x in ibd_list]
    k = len(ibd_list)

    # Make List of geographic Position of Individuals. Sort them so that same pair of INDs always in the same order!
    geo_inds = [min(x[2], x[3]) + max(x[2], x[3]) for x in ibd_list]    

    def unique_rows(data):
        '''Gives back unique rows in data and the indices needed to reconstruct the original thing'''
        data = np.copy(data)  # Make copy to avoid GC getting stuck.
        uniq, indices = np.unique(data.view(data.dtype.descr * data.shape[1]), return_inverse=True)
        return np.copy(uniq.view(data.dtype).reshape(-1, data.shape[1])), np.copy(indices)  

    _, inds = unique_rows(geo_inds)  # Extract "unique" Indices
    nr_unique_prs = np.max(inds)+1     # The Nr of unique pairs (including 0)

    bl_ls = [[] for _ in xrange(nr_unique_prs)] # List of Lists for IBD-Blocks

    for i in xrange(len(inds)):
        ind = inds[i] # Get Index
        bl_ls[ind].append(ibd_list[i]) # Append the block to its unique Position.

    def merge_blocks(block_ls, spacing):
            '''Merge blocks between Individuals.
            spacing: Maximal spacing of blocks to be fused.
            block_ls: List of blocks - [[start, end, time]
            I.: First detect all same pairs that share IBD and pool their blocks.
            II.: Then merge these blocks if needed.'''
            assert spacing >= 0  # Sanity Check
    
            block_ls.sort()  # Sort blocks by Start Point
             
            block_ls_final = []  # Empty Container for the final Block List
            start, end, t = block_ls[0]  # Temporary Variables
            
            for bl in block_ls[1:]:
                if (bl[0] - end) < spacing:  # If Overlap
                    t = min(t, bl[2]) # Set time to minimum
                    end = max(bl[1], end)  # Extend
                    
                else:
                    block_ls_final.append((start, end, t))  # Append another Block
                    start = bl[0]
                    end = bl[1]
                    t = bl[2]
            block_ls_final.append((start, end, t))  # Append another Block
            return block_ls_final
        
        
    for blocks in bl_ls:
        input_ls = [[x[0], x[1], x[4]] for x in blocks] # Extract list of block Starts and Ends.
        blocks_final = merge_blocks(input_ls, spacing) # Do the Merging
        bl=blocks[0]

        for start, end, t in blocks_final:
            bl_list_final.append((start, end, bl[2], bl[3], t)) 


    # Restore 2nd entry to relative length of block:
    bl_list_final = [(x[0], x[1] - x[0], x[2], x[3], x[4]) for x in bl_list_final]
    end = timer()
    print(start)
    print(end)
    print("Time for Postprocessing: %.5f s" % (end - start0))
    print("Merged from %i to %i IBD blocks." % (k, len(bl_list_final)))
    IBD_blocks = bl_list_final
    return IBD_blocks

In [14]:
fake_ibd_list=[(0.1, 0.03, (3,4), (2,4), 6),(0.2, 0.5, (1,1), (2,2), 10),(0.15, 0.4, (2,4), (3,4), 9), (0.3, 0.6, (2,4), (3,4), 9)]
fake_ibd_list

[(0.1, 0.03, (3, 4), (2, 4), 6),
 (0.2, 0.5, (1, 1), (2, 2), 10),
 (0.15, 0.4, (2, 4), (3, 4), 9),
 (0.3, 0.6, (2, 4), (3, 4), 9)]

In [19]:
post_process_IBD(fake_ibd_list, 0.02)

Starting Post Processing!
0.1
1517845591.56
Time for Postprocessing: 0.00061 s
Merged from 4 to 2 IBD blocks.


[(0.2, 0.49999999999999994, (1, 1), (2, 2), 10),
 (0.1, 0.7999999999999999, (3, 4), (2, 4), 6)]

[(1, 4)]

In [116]:
def unique_rows(data):
    '''Gives back unique rows in data and the indices needed to reconstruct the original thing'''
    data = np.copy(data)  # Make copy to avoid GC getting stuck.
    uniq, indices = np.unique(data.view(data.dtype.descr * data.shape[1]), return_inverse=True)
    return np.copy(uniq.view(data.dtype).reshape(-1, data.shape[1])), np.copy(indices)   

[(2, 3, 3, 4), (3, 3, 3, 4), (2, 3, 3, 4)]

In [11]:
l=np.array([1,2,3])
l

array([1, 2, 3])

In [17]:
a=fake_ibd_list
a

[(0.1, 0.03, (3, 4), (2, 4), 6),
 (0.2, 0.5, (1, 1), (2, 2), 10),
 (0.15, 0.4, (2, 4), (3, 4), 9),
 (0.3, 0.6, (2, 4), (3, 4), 9)]

In [27]:
a=None
if a:
    print("lul")